In [1]:
parent_folder = '/Users/username/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/username/Desktop/ML/Recommendations/arcane/')

from src._utils import load_bertopic_model_from_hf

/Users/username/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/username/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/username/anaconda3/envs/recoenv/lib/python3.11/site-pac

In [2]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [3]:
with PostgresDatabaseOperation() as cursor:
    sql = '''SELECT feed_id, uai.article_id, acm.cluster_id, parent_name, uai.created_at 
    FROM user_article_interactions uai 
    LEFT JOIN article_to_cluster_mapping acm ON uai.article_id = acm.article_id
    LEFT JOIN cluster_hierarchy ch ON acm.cluster_id = ch.parent_id
    WHERE user_id = 40 AND uai.created_at > '2023-11-07'
    '''
    cursor.execute(sql)
    results = cursor.fetchall()

In [4]:
import pandas as pd

In [5]:
df = pd.DataFrame(results, columns = ['feed_id', 'article_id', 'cluster_id', 'cluster_name', 'created_at'])

In [6]:
df

,feed_id,article_id,cluster_id,cluster_name,created_at
0,578,652fcfad1e5cc42b1b13a357,316,loan_card_insurance_credit card_score,2023-11-07 11:30:08.555677
1,577,652fd1791e5cc42b1b13a43a,310,sebi_commodity_fo_ban_contracts,2023-11-07 11:29:21.176194
2,577,652fd35d1e5cc42b1b13a529,294,yes bank_yes_yoy_npa_net profit,2023-11-07 11:29:05.357844
3,577,652cacea50af0e25a91230ca,294,yes bank_yes_yoy_npa_net profit,2023-11-07 11:30:00.050633
4,578,65309bf51e5cc42b1b13a5f8,175,lt_lt stock_price lt_data lt_net change,2023-11-07 11:30:13.196002
5,577,6531693d1e5cc42b1b13d761,316,loan_card_insurance_credit card_score,2023-11-07 11:29:05.359223
6,577,653168fc1e5cc42b1b13c720,264,airline_aircraft_air_aviation_airlines,2023-11-07 11:28:58.096475
7,578,65316d4a1e5cc42b1b13e22c,266,people_bjp_congress_party_drugs,2023-11-07 11:30:05.238066
8,577,65316f411e5cc42b1b13f5e4,310,sebi_commodity_fo_ban_contracts,2023-11-07 11:29:33.689144
9,578,65316f541e5cc42b1b13f9d1,198,live blog_blog_price live_52 week_2023 mint,2023-11-07 11:30:10.895546


In [7]:
from src.articles.ArticleService import ArticleService

In [8]:
article_ids = df['article_id'].unique().tolist()

In [9]:
articles = ArticleService.parallel_get_Articles(article_ids)

In [10]:
titles = pd.DataFrame([(art_id, article.title, article.url) for art_id, article in articles.items()], columns = ['article_id', 'title', 'url'])

In [11]:
titles['category'] = titles['url'].apply(lambda x: 'wint' if 'wint' in x else 'groww' if 'groww' in x else x.split('.com/')[1].split('/')[0] if 'livemint' in x else None)

In [12]:
titles.groupby('category')['article_id'].count()

category
budget        1
companies     1
groww         1
industry      1
market       10
opinion       1
wint          3
Name: article_id, dtype: int64

In [13]:
titles.url.apply(lambda x: x.split('.com')[0]).unique()

array(['https://www.livemint', 'https://www.wintwealth',
       'https://groww.in/blog/state-bank-of-india-quarterly-results-q3-fy22'],
      dtype=object)

In [14]:
df = pd.merge(df, titles)

In [15]:
df

,feed_id,article_id,cluster_id,cluster_name,created_at,title,url,category
0,578,652fcfad1e5cc42b1b13a357,316,loan_card_insurance_credit card_score,2023-11-07 11:30:08.555677,How to Expand Your Operations with a Business ...,https://www.wintwealth.com/blog/how-to-expand-...,wint
1,577,652fd1791e5cc42b1b13a43a,310,sebi_commodity_fo_ban_contracts,2023-11-07 11:29:21.176194,"Stock Market Trading: Meaning, History, Types ...",https://www.wintwealth.com/blog/stock-market-t...,wint
2,577,652fd35d1e5cc42b1b13a529,294,yes bank_yes_yoy_npa_net profit,2023-11-07 11:29:05.357844,Indiabulls Housing Finance September 2023 Publ...,https://www.wintwealth.com/blog/indiabulls-hou...,wint
3,577,652cacea50af0e25a91230ca,294,yes bank_yes_yoy_npa_net profit,2023-11-07 11:30:00.050633,Punjab National Bank Share Price Live blog for...,https://www.livemint.com/market/live-blog/punj...,market
4,578,65309bf51e5cc42b1b13a5f8,175,lt_lt stock_price lt_data lt_net change,2023-11-07 11:30:13.196002,L&T Technology Share Price Live blog for 19 Oc...,https://www.livemint.com/market/live-blog/lt-t...,market
5,577,6531693d1e5cc42b1b13d761,316,loan_card_insurance_credit card_score,2023-11-07 11:29:05.359223,UPI transactions surged to 9.3 billion in June...,https://www.livemint.com/industry/banking/upi-...,industry
6,577,653168fc1e5cc42b1b13c720,264,airline_aircraft_air_aviation_airlines,2023-11-07 11:28:58.096475,Airlines carry record 12.9 million passengers ...,https://www.livemint.com/companies/news/airlin...,companies
7,578,65316d4a1e5cc42b1b13e22c,266,people_bjp_congress_party_drugs,2023-11-07 11:30:05.238066,A tragedy of commons shouldn’t need a tipping ...,https://www.livemint.com/opinion/online-views/...,opinion
8,577,65316f411e5cc42b1b13f5e4,310,sebi_commodity_fo_ban_contracts,2023-11-07 11:29:33.689144,"GMR Infra, 6 others placed under F&O ban on NS...",https://www.livemint.com/market/stock-market-n...,market
9,578,65316f541e5cc42b1b13f9d1,198,live blog_blog_price live_52 week_2023 mint,2023-11-07 11:30:10.895546,ICICI securities initiates buy call on KFIN Te...,https://www.livemint.com/market/stock-market-n...,market


In [54]:
df.groupby('cluster_name')['article_id'].count()

cluster_name
air_flight_airport_air india_india           2
bank_rbi_loan_credit_banks                  10
coal_steel_mt_coal india_tata steel          2
crypto_target price_price rs_bitcoin_buy    11
dollar_rupee_paise_dollar rupee_82           2
drug_usfda_gets_pharma_gets usfda            4
firms_mcap_valued_valued firms_10 valued     2
gold_silver_rs_results_profit                4
gst_tax_pli_gst council_dpiit                7
inflation_growth_rate_fed_fy24               9
ipo_adani_papers_papers sebi_power          12
irdai_life_insurance_insurers_lic            6
oil_gas_ongc_energy_transition               6
pension_epfo_higher_eps_eps pension          8
projects_cr_fpis_rs_equities                 2
stocks_52 week_52_week_nifty                 8
trade_exports_uk_fta_india                   3
vande_vande bharat_train_railway_bharat      2
Name: article_id, dtype: int64

In [57]:
df[df.category == 'market'].groupby('feed_id')['article_id'].count()

feed_id
565    1
566    3
568    4
569    2
570    5
571    5
572    1
573    3
574    8
575    6
Name: article_id, dtype: int64

In [60]:
df[df.category == 'market'].article_id.count()

38

In [63]:
len(df[df.feed_id.isin([574, 575])])

20

In [61]:
len(df)

100

In [59]:
df[df.category == 'market'].groupby('cluster_name')['article_id'].count()

cluster_name
coal_steel_mt_coal india_tata steel         1
crypto_target price_price rs_bitcoin_buy    7
dollar_rupee_paise_dollar rupee_82          2
firms_mcap_valued_valued firms_10 valued    2
gold_silver_rs_results_profit               2
inflation_growth_rate_fed_fy24              3
ipo_adani_papers_papers sebi_power          9
oil_gas_ongc_energy_transition              3
projects_cr_fpis_rs_equities                1
stocks_52 week_52_week_nifty                8
Name: article_id, dtype: int64

In [56]:
df[df.feed_id == 570]

,feed_id,article_id,cluster_id,cluster_name,created_at,title,url,category
30,570,653169111e5cc42b1b13cd32,182,drug_usfda_gets_pharma_gets usfda,2023-11-06 12:10:43.146305,HDFC AMC is launching a fresh edition of its C...,https://www.livemint.com/money/personal-financ...,money
49,570,65316f171e5cc42b1b13ef0a,126,irdai_life_insurance_insurers_lic,2023-11-06 12:11:01.445066,Best health insurance plans for you | Mint,https://www.livemint.com/Money/bcy1XPInx2ZTgwr...,Money
56,570,65316f731e5cc42b1b1400ad,191,crypto_target price_price rs_bitcoin_buy,2023-11-06 12:10:46.824794,"Bajaj Finserv closed today at ₹1412.8, down -0...",https://www.livemint.com/market/live-blog/baja...,market
62,570,65316fb61e5cc42b1b141418,192,ipo_adani_papers_papers sebi_power,2023-11-06 12:11:06.804855,Union Bank of India announces fundraising plan...,https://www.livemint.com/market/stock-market-n...,market
75,570,65316fee1e5cc42b1b14292b,206,inflation_growth_rate_fed_fy24,2023-11-06 12:10:48.027676,Global markets update: US stocks mixed ahead o...,https://www.livemint.com/market/stock-market-n...,market
76,570,65316ff51e5cc42b1b142c4a,191,crypto_target price_price rs_bitcoin_buy,2023-11-06 12:10:58.544887,Sbi Life share price Today Live Updates : Sbi ...,https://www.livemint.com/market/live-blog/sbi-...,market
80,570,65367fcb1e5cc42b1b143bd7,126,irdai_life_insurance_insurers_lic,2023-11-06 12:10:46.904295,SBI Life Insurance Q4 FY22 Results - 84.6% Jum...,https://groww.in/blog/sbi-life-insurance-q4-fy...,groww
86,570,65367ff21e5cc42b1b143d2b,182,drug_usfda_gets_pharma_gets usfda,2023-11-06 12:10:48.024844,Analysis of the Pharma Sector in India,https://groww.in/blog/indian-pharma-sector-ana...,groww
95,570,65316efb1e5cc42b1b13eb4d,206,inflation_growth_rate_fed_fy24,2023-11-06 12:11:07.465208,RBI signals it’s now at cruising altitude | Mint,https://www.livemint.com/opinion/columns/rbi-s...,opinion
99,570,653884dc1e5cc42b1b14443d,192,ipo_adani_papers_papers sebi_power,2023-11-06 12:10:43.144520,Powergrid Share Price Live blog for 25 Oct 202...,https://www.livemint.com/market/live-blog/powe...,market
